In [1]:
from pathlib import Path
import fasttext
from fasttext import util as ftu

In [3]:
vec_file = Path("/Users/stm/cltk_data/cc.la.300.bin")

In [4]:
vec_file.exists()

True

In [5]:
embeddings = fasttext.load_model(str(vec_file))

In [11]:
embeddings.get_dimension()

300

In [14]:
ftu.reduce_model(embeddings, 100)

In [6]:
embeddings.get_dimension()

100

In [16]:
embeddings.save_model(str(vec_file).replace("300", "100"))

In [7]:
from latin_author_learning.fasttext_wrapper import model_to_vec_str

/Users/stm/Library/Caches/pypoetry/virtualenvs/latin-author-learning-nqRTE_o3-py3.9/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [8]:
vec_str = model_to_vec_str(embeddings)

In [2]:
from pathlib import Path
import os

In [3]:
cltk_path = Path(os.environ["HOME"]) / "cltk_data" / "lat" / "text"

In [4]:
perseus_path = cltk_path / "lat_text_perseus"

In [13]:
vec_path = cltk_path / "cc.la.100.vec"

In [13]:
with open(vec_path, "w") as f:
    f.write(vec_str)

In [5]:
from latin_author_learning.corpus import Corpus

In [6]:
perseus = Corpus("lat_text_perseus")

In [7]:
perseus.add_data_from_files(
    perseus_path,
    filename_contains="_lat.",
    meta_keys=["fileDesc", "teiHeader"],
    meta_key_prefix="@", 
)

In [8]:
from latin_author_learning.fasttext_wrapper import DatasetWrapper

/Users/stm/Library/Caches/pypoetry/virtualenvs/latin-author-learning-nqRTE_o3-py3.9/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [9]:
ds = DatasetWrapper(corpus=perseus, fraction_for_test=0.2)

In [10]:
trainfile = cltk_path / "perseues_train.txt"
ds.get_training_data(trainfile)

In [11]:
valid_file = cltk_path / "perseues_valid.txt"
ds.get_validation_data(valid_file)

In [16]:
author_model = fasttext.train_supervised(
    input=str(trainfile), dim=100, lr=1., thread=6,
    pretrainedVectors=str(vec_path),
    epoch=1000,
)

Read 3M words
Number of words:  392447
Number of labels: 23
Progress: 100.0% words/sec/thread: 2958989 lr:  0.000000 avg.loss:  0.051756 ETA:   0h 0m 0s


In [17]:
author_model.test(str(valid_file))

(43, 0.8837209302325582, 0.8837209302325582)